In [25]:
!pip install wget
!pip install unicodecsv

In [0]:
from bs4 import BeautifulSoup
import wget
import requests
import datetime
from datetime import date
import pandas as pd
import re

In [0]:
total_articles = [] # it will save all publications

In [91]:
#Fechas en las cuales se buscara información en la página perú 21
num_years = 1/4; #numero de años a evaluar
iterator_day = datetime.date.today() #fecha actual de consulta
last_publication = iterator_day - datetime.timedelta(days=int(365*num_years)) #Ultima fecha de consulta

print(iterator_day, last_publication)

2019-09-23 2019-06-24


In [0]:
#Se crea un array con los días a iterar
array_days = []
i = iterator_day
while(i > last_publication):
    array_days.append(str(i))
    i = i - datetime.timedelta(days=1)

## 0. Pruebas funcionamiento

### Prueba en una sola página
Solo se ingresa a la página y se obtinenen todas las URL de los articulos

In [30]:
#Primero definimos la URL de la página a analizar
day = datetime.date(2019, 9, 20)
url_day = "https://peru21.pe/archivo/todas/"+str(day)

content_day = requests.get(url_day).content
soup_day = BeautifulSoup(content_day, 'lxml')
article_url = soup_day.findAll('a',{'class' : 'story-item__title'}) #Busca todos los titulos en la pagina web de consulta
print(url_day)

https://peru21.pe/archivo/todas/2019-09-20


In [0]:
for i in article_url:
  print(i['href'])

###Prueba de scrapping en un solo articulo
Se busca la informacion necesaria

In [65]:
url_article = 'https://peru21.pe/espectaculos/internacional/veronica-castro-este-es-el-momento-que-hizo-click-con-yolanda-andrade-noticia/' #'https://peru21.pe/deportes/futbol-internacional/premier-league-programacion-partidos-fecha-6-liga-inglesa-nczd-501959-noticia/' #'https://peru21.pe/peru/senamhi-fuertes-vientos-y-granizada-en-chanchamayo-son-por-una-tormenta-localizada-noticia/'
content_article = requests.get(url_article).content
soup_article = BeautifulSoup(content_article, 'lxml')
#soup_article
#title = soup_article.find('h1', {'class' : 'story-header__news-title'})
#summary = soup_article.find('h2', {'class' : 'story-header__news-summary'})
#category = soup_article.find('div', {'class' : 'text-gray-300'})
#author = soup_article.find('a', {'class' : 'secondary-font'})
#date_art = soup_article.find('time', {})
#art_content = soup_article.findAll('p', {'class' : 'story-content__font--secondary'})
#art = [i.text for i in art_content]
#sep = ' '
#a = sep.join(art)
#print(title.text)
#print(summary.text)
#print(category.text)
#print(author.text)
#print(date_art.text)
#print(a)

 Este es el momento en el que Verónica Castro hizo clic con Yolanda Andrade
 Video muestra la primera vez que la conductora y la actriz estuvieron juntas ante cámaras y revela las románticas actitudes entre ambas.
Espectáculos Internacional
Redacción PERÚ21
Actualizado en 23/09/2019 a las 21:37
Luego del escándalo suscitado por las revelaciones de Yolanda Andrade, respecto a una supuesta boda con Verónica Castro, son muchas las imágenes que evidenciarían la aparente relación que existió entre ambas. Sin embargo, dentro de todo el material mostrado por los medios de espectáculos, se reveló el video de una entrevista realizada en 2003 por Andrade —para su programa ‘Hijas de la Madre Tierra’— en la que se habría desencadenado esta historia de amor. En las imágenes, compartidas por el programa argentino 'Hay que ver’, se ve a una Yolanda Andrade embelesada por cada una de las respuestas de la actriz mexicana y, en algunos instantes, le acaricia la pierna mientras le sonríe. En otro momento

## 1. Funciones 

### 1.1 Funciones para adquirir informacion del articulo

In [0]:
def get_title(soup_article):
  #Elemento tag que busca la linea HTML relacionada al titulo
  tag = soup_article.find('h1', {'class' : 'story-header__news-title'}) 
  #Se verifica que lo obtenido no sea del tipo None
  if tag != None:
    titulo = tag.text
  else:
    titulo = None
  
  return titulo 

def get_summary(soup_article):
  #Elemento tag que busca la linea HTML relacionada al resumen
  tag = soup_article.find('h2', {'class' : 'story-header__news-summary'})
  #Se verifica que lo obtenido no sea del tipo None
  if tag != None:
    resumen = tag.text
  else:
    resumen = None
  
  return resumen 

def get_category(soup_article):
  #Elemento tag que busca la linea HTML relacionada a la categoria
  tag = soup_article.find('div', {'class' : 'text-gray-300'})
  #Se verifica que lo obtenido no sea del tipo None
  if tag != None:
    categoria = tag.text
  else:
    categoria = None
  
  return categoria

def get_category(soup_article):
  #Elemento tag que busca la linea HTML relacionada a la categoria
  tag = soup_article.find('div', {'class' : 'text-gray-300'})
  #Se verifica que lo obtenido no sea del tipo None
  if tag != None:
    categoria = tag.text
  else:
    categoria = None
  
  return categoria 

def get_author(soup_article):
  #Elemento tag que busca la linea HTML relacionada al autor
  tag = soup_article.find('a', {'class' : 'secondary-font'})
  #Se verifica que lo obtenido no sea del tipo None
  if tag != None:
    autor = tag.text
  else:
    autor = None
  
  return autor 

def get_date(soup_article):
  #Elemento tag que busca la linea HTML relacionada al autor
  tag = soup_article.find('time')
  #Se verifica que lo obtenido no sea del tipo None
  if tag != None:
    txt = tag.text
    #print(txt)
    fecha = re.findall(r'\d{2}/\d{2}/\d{4}',txt) #Se obtiene la fecha del string
    hora = re.findall(r'\d{2}:\d{2}',txt) #Se obtiene la hora del string
    fe = fecha[0]+' '+hora[0] #Se unen los datos para convertir a datetime
    fecha_hora = datetime.datetime.strptime(fe, '%d/%m/%Y %H:%M')
    fecha_hora = fecha_hora - datetime.timedelta(hours=5)
    #print(fecha_hora)
  else:
    fecha_hora = None
  
  return fecha_hora 

def get_contenido(soup_article):
  #Elemento tag que busca la linea HTML relacionada al autor
  tag = soup_article.findAll('p', {'class' : 'story-content__font--secondary'})
  #Se verifica que lo obtenido no sea del tipo None
  if tag != None:
    art = [i.text for i in tag]
    sep = ' '
    full_content = sep.join(art)
  else:
    full_cotent = None
  
  return full_content

### Funcion para adquirir informacion de varios días

In [0]:
def obtenerArticulos(array_days):
  pd_articulos = pd.DataFrame()
  #pd_articulos
  lst_titulos = []
  lst_resumenes = []
  lst_categorias = []
  lst_autores = []
  lst_fechas = []
  lst_contenidos = []

  for dia in array_days:
    url_day = 'https://peru21.pe/archivo/todas/'+str(dia) #Ingreso al día
    content_day = requests.get(url_day).content
    soup_day = BeautifulSoup(content_day, 'lxml')
    article_url = soup_day.findAll('a',{'class' : 'story-item__title'}) #Busca todos los titulos en la pagina web de consulta

    for i in article_url:
      url_art = 'https://peru21.pe'+i['href']
    
      content_article = requests.get(url_art).content
      soup_article = BeautifulSoup(content_article, 'lxml')
    
      title = get_title(soup_article)
      summary = get_summary(soup_article)
      category = get_category(soup_article)
      author = get_author(soup_article)
      date_art = get_date(soup_article)
      
      full_content = get_contenido(soup_article)

      lst_titulos.append(title)
      lst_resumenes.append(summary)
      lst_categorias.append(category)
      lst_autores.append(author)
      lst_fechas.append(date_art)
      lst_contenidos.append(full_content)

  pd_articulos['Fecha'] = lst_fechas
  pd_articulos['Categoria'] = lst_categorias 
  pd_articulos['Autor'] = lst_autores
  pd_articulos['Titulo'] = lst_titulos
  pd_articulos['Resumen'] = lst_resumenes
  pd_articulos['Contenido'] = lst_contenidos

  return pd_articulos

## 2. Prueba Obtención de Articulos

In [97]:
pd_art = obtenerArticulos(array_days)

KeyboardInterrupt: ignored

In [98]:
pd_art.head(10)

,Fecha,Categoria,Autor,Titulo,Resumen,Contenido
0,2019-09-23 17:27:00,Farándula,Redacción PERÚ21,Josimar cantó “La protagonista” en fiesta que...,La joven empresaria llegó a un acuerdo con el...,Josimar cumplió su palabra y se presentó en la...
1,2019-09-23 17:23:00,Fútbol Internacional,Redacción PERÚ21,FIFA The Best: Estas son las jugadoras que in...,"La capitana del equipo de Estados Unidos, Meg...",Este lunes se realizó la premiación FIFA The B...
2,2019-09-23 17:21:00,Series,Redacción PERÚ21,Actores de “Friends” reciben US$20 millones a...,"A 25 años de su estreno, Warner Bros ingresa ...",“Friends” cumplió 25 años y sigue dando que ha...
3,2019-09-23 17:18:00,Fútbol Internacional,Redacción PERÚ21,Liberman sobre el The Best de Messi: “Lo mere...,El periodista argentino cuestionó la elección...,Lionel Messi ganó por primera vez el premio Th...
4,2019-09-23 17:16:00,Mundo,Redacción PERÚ21,Senador hiere a fotógrafo tras disparar contr...,Jean Marie Ralph Fethière realizó disparos e ...,Un fotoperiodista de la agencia AP resultó her...
5,2019-09-23 17:13:00,Farándula,Redacción PERÚ21,¡Otra faceta! Macarena Vélez alista su debut...,La ‘chica reality’ compartió la noticia a tra...,La 'guerrera’ Macara Vélez decidió dejar atrás...
6,2019-09-23 17:09:00,Espectáculos Internacional,Redacción PERÚ21,Tríos sexuales e infidelidades: Historia de d...,La protagonista de Ghost sorprendió al contar...,Demi Moore se separó en 2011 de Asthon Kutcher...
7,2019-09-23 16:37:00,Espectáculos Internacional,Redacción PERÚ21,Este es el momento en el que Verónica Castro ...,Video muestra la primera vez que la conductor...,Luego del escándalo suscitado por las revelaci...
8,2019-09-23 16:13:00,La voz del 21,None,Omar Cairo: Congreso recibe a Comisión de Ven...,None,
9,2019-09-23 16:00:00,Espectáculos Internacional,Redacción PERÚ21,Maluma vuelve a presumir su marcado abdomen e...,La fotografía que publicó el cantante colombi...,"¡Remeció las redes! Maluma, en su cuenta de In..."


In [94]:
pd_art.tail(10)

,Fecha,Categoria,Autor,Titulo,Resumen,Contenido
190,2019-09-22 07:10:00,Política,Redacción PERÚ21,La aprobación de las leyes: cuando todos quie...,Identificamos los dictámenes aprobados por el...,"Hace algunos meses, a través del proyecto Parl..."
191,2019-09-22 07:10:00,Policiales,Shirley Avila,Descuartizados de San Martín de Porres | Test...,Testigos revelan posible móvil de macabro cri...,“Por bruja (en Venezuela este término signific...
192,2019-09-22 07:10:00,Política,Michael Machacuay,Bancadas evalúan pedir la nulidad del proceso...,"Peruanos por el Kambio, Unidos por la Repúbli...",La nulidad del proceso de selección de seis ma...
193,2019-09-22 07:10:00,Economía,Mariana Ruiz,Canal online sostendrá la campaña navideña,Los juguetes que más venden los retails son ...,"La campaña navideña será mejor que la de 2018,..."
194,2019-09-22 07:05:00,Salud,Redacción PERÚ21,Salud21: Contagio de la varicela aumenta en p...,"Ante la sospecha de un niño con varicela, los...",Este 23 de setiembre empieza la primavera. El ...
195,2019-09-22 07:05:00,Política,Ivan Atilano,Comisión de Educación pone trabas a informe d...,Rechazaron un reporte de 796 hojas sobre la g...,"El último lunes, la presidenta de la Comisión ..."
196,2019-09-22 07:00:00,Perú,None,Lambayeque: No gastaron recursos para áreas n...,Defensoría del Pueblo efectuó supervisión y s...,"En Lambayeque, los hospitales Regional, Las Me..."
197,2019-09-22 07:00:00,Pronósticos y realidades,Ricardo V. Lago,La semana en que tanto dinero cundió tan poco,“Esta semana hubo una crisis de liquidez. Per...,“Me emborraché para olvidarte y ahora te veo d...
198,2019-09-22 07:00:00,Ariel Segal,Ariel Segal,Contexto de un ataque (I),None,El ataque con drones a instalaciones petrolera...
199,2019-09-22 07:00:00,Cultura,Pierina Denegri,Todo sobre el espectáculo ‘Cuéntame’ de Pedro...,El músico y cantante peruano presenta pasajes...,En su niñez supo que quería ser cantante graci...


In [0]:
pd_art.to_csv('prueba1.csv',encoding='utf_8')